In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from banditrec.datasets import parse_yahoo2
from banditrec.simulate import simulate_replay
from banditrec.agents import RandomAgent, HindsightAgent, UCBAgent, CLUBAgent, NUCBAgent, SCLUBAgent, DynUCBAgent
from banditrec.experiments import ExperimentVisualizer

## Load dataset

In [ ]:
DATA_BASE_PATH = "./data/yahoo2"

# load raw
DATA_PATHS = [f'{DATA_BASE_PATH}/ydata-fp-td-clicks-v2_0.201110{i:0>2}' for i in range(2, 16+1)]
e, u, p = parse_yahoo2(DATA_PATHS)

# save processed
e.to_parquet("./data/yahoo2/all.parquet")
np.savez_compressed('./data/yahoo2/metadata.npz', user_features=u, pool_masks=p)

# load processed
e = pd.read_parquet("./data/yahoo2/all.parquet")
data = np.load("./data/yahoo2/metadata.npz")
u = data["user_features"]
p = data["pool_masks"].astype(bool)

In [ ]:
# get 18k version of dataset
t = 1607525 + 2220905 + 2189981 + 1373877 + 1921382 + 2040503 + 1571772 # first week
e_eval = e.iloc[:t].copy()
e_eval = e_eval[e_eval.user != 4] # null user
users = e_eval.user.value_counts()
users_18k = users[users>=50]
e18k = e_eval[e_eval.user.isin(users_18k.index)].copy()
e18k["user"] = e18k["user"].factorize()[0]
e18k["item"] = e18k["item"].factorize()[0]

ranked_items = e18k[e18k.clicked].item.value_counts().index.values

## Run experiment

In [ ]:
results = []
for agent in [
    RandomAgent(),
    HindsightAgent(ranked_items),
    UCBAgent(0.12),
    CLUBAgent(0.08, 2),
    DynUCBAgent(0.1, 1),
    SCLUBAgent(0.09, 2, 2),
    NUCBAgent(0.03, 100, 0)
]:
    res = simulate_replay(agent, e18k, p)
    results.append(res)

In [ ]:
# save results
time_horizon = 71000
output_path = "results/raw"

def bucketize(array, bucket_count=500):
    bucket_size = len(array)  // bucket_count
    return array[:bucket_count * bucket_size].reshape(bucket_count, -1).sum(axis=1)

def save_result(result, name):
    rew = result.rewards[:time_horizon]
    b = bucketize(rew)
    np.save(f"{output_path}/{name}.npy", b)

save_result(results[0], "Random")
save_result(results[1], "Hindsight")
save_result(results[2], "UCB")
save_result(results[3], "CLUB")
save_result(results[4], "DynUCB")
save_result(results[5], "SCLUB")
save_result(results[6], "NUCBAgent")

## Create plots

In [ ]:
md = 12
vis = ExperimentVisualizer("results/raw", "results/plots")
vis.new_plot(None, time_horizon)
vis.add_result("Random",    {"color": "tab:blue"})
vis.add_result("UCB",     {"color": "tab:green",  "marker": "v", "markevery": (0*md, 5*md)})
vis.add_result("CLUB",    {"color": "tab:red",    "marker": "^", "markevery": (1*md, 5*md)})
vis.add_result("DynUCB",  {"color": "tab:brown",  "marker": "<", "markevery": (2*md, 5*md)})
vis.add_result("SCLUB",   {"color": "tab:gray",   "marker": ">", "markevery": (3*md, 5*md)})
vis.add_result("NUCB",    {"color": "tab:purple", "marker": "o", "markevery": (4*md, 5*md)})
vis.create_plot("ctrs")